<a href="https://colab.research.google.com/github/mikkelgolf/testrepo/blob/main/Golfbox_tournament_schedule_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade pip
!pip install playwright pandas
!playwright install chromium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [playwright]
167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 23.9s167.7 MiB [] 0% 9.8s167.7 MiB [] 1% 4.4s167.7 MiB [] 2% 3.2s167.7 MiB [] 3% 2.6s167.7 MiB [] 4% 2.3s167.7 MiB [] 5% 2.1s167.7 MiB [] 5% 2.2s167.7 MiB [] 6% 2.1s167.7 MiB [] 7% 2.1s167.7 MiB [] 8% 2.0s167.7 MiB [] 9% 2.0s167.7 MiB [] 10% 1.9s167.7 MiB [] 11% 1.8s167.7 MiB [] 12% 1.8s167.7 MiB [] 13% 1.7s167.7 MiB [] 14% 1.7s167.7 MiB [] 15% 1.6s167.7 MiB [] 17% 1.6s167.7 MiB [] 18% 1.5s167.7 MiB [] 19% 1.5s167.7 MiB [] 20% 1.5s167.7 MiB [] 21% 1.5s167.7 MiB [] 22% 1.4s167.7 MiB [] 23% 1.4s167.7 MiB [] 25% 1.3s167.7 MiB [] 26% 1.3s167.7 MiB [] 27% 1.3s167.7 MiB [] 28% 1.3s167.7 MiB [] 29% 1.2s1

In [13]:
import pandas as pd
import re
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from IPython.display import FileLink

nest_asyncio.apply()  # Needed to run asyncio in Colab

async def extract_tournaments():
    tournaments = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        await page.goto('https://danish.golf/ecco-tour/livescore/#/schedule', timeout=60000)
        await page.wait_for_timeout(7000)  # Wait for JS content to load

        a_tags = await page.query_selector_all('a.competitionName')
        for a in a_tags:
            try:
                # Name
                div = await a.query_selector('div.truncate')
                name = (await div.get_attribute('title')).strip() if div else ''

                # Competition ID from onclick
                onclick = await a.get_attribute('onclick')
                comp_id_match = re.search(r'onCompetitionClick\((\d+)\)', onclick or '')
                comp_id = comp_id_match.group(1) if comp_id_match else ''

                base_url = 'https://golfbox.no/livescoring/tour/?language=2057#/competition/'
                livescoring_link = f"{base_url}{comp_id}/leaderboard"
                players_link = f"{base_url}{comp_id}/players"

                # --- NEW: Open players link and extract date and course ---
                course = ""
                comp_date = ""
                try:
                    page2 = await context.new_page()
                    await page2.goto(players_link, timeout=30000)
                    await page2.wait_for_timeout(2000)  # wait a little for content

                    # Dates
                    date_elem = await page2.query_selector('div.competition-date')
                    comp_date = (await date_elem.inner_text()).strip() if date_elem else ""

                    # Golf Course
                    course_elem = await page2.query_selector('div.competition-subtitle')
                    course = (await course_elem.inner_text()).strip() if course_elem else ""

                    await page2.close()
                except Exception as e:
                    print(f"Could not fetch extra details for {name}: {e}")

                tournaments.append({
                    'Tournament Name': name,
                    'Tournament Dates': comp_date,  # from players page (reliable)
                    'Golf Course': course,
                    'Players List Link': players_link,
                    'Livescoring Link': livescoring_link
                })
            except Exception as e:
                print("Skipped one:", e)
        await browser.close()
    return tournaments

# Run the async extraction in Colab
data = asyncio.run(extract_tournaments())
df = pd.DataFrame(data)
csv_filename = 'golfbox_tournaments.csv'
df.to_csv(csv_filename, index=False)
FileLink(csv_filename)
from google.colab import files
files.download('golfbox_tournaments.csv')




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>